In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Sample data provided
data = [
    {"user_id": None, "product_id": "B09TRF4LM9", "review_id": "R100AEOXPX3ZP7", "rating": 5.0,
     "review_text": "wasn't sure about an off-brand, but this charger works great! Doesn't overheat and charges very rapidly.",
     "sentiment_id": None},
    {"user_id": None, "product_id": "B09BG6DSBY", "review_id": "R101PRE3GXWPG7", "rating": 2.0,
     "review_text": "These are so slow!! Even with an extra storage card, they run so slow.", "sentiment_id": None},
    {"user_id": None, "product_id": "B08438H4GM", "review_id": "R102U9PYQ6S094", "rating": 5.0,
     "review_text": "So I've tried several different docks and this seems to be the best for my new MacBook Air M1. I wanted to write a review because everybody is reviewing for Pro's and the Air is different BECAUSE of the power adapter that comes with it. The Air comes with a 30 watt power adapter so when I would plug it in through the dock it would not charge. The work around is to get a 60 watt power adapter which is also still compatible with the Air. The issue is the dock uses some of the power and not all of the 30 watts goes through to the computer and what little that did go through the computer did not recognize as being enough to charge. The 60 watt is working great and don't worry, I did verify all of this with both Apple and Belkin tech support. I'm using one very large external monitor (this doesn't work with dual monitors on the M1...not a Belkin thing it's and apple thing) both USB ports for various things and the 3.5mm sound out to a mixer/speakers and the charger for the computer.  It is really convenient as I only have to plug in the one thing into the thunderbolt and everything is up and running. The cable is short but I use my computer as a second screen and have a raised docking station for it to be at eye level. I mounted this dock to the back of that stand in the upper left so the connector just literally loops around and in. Very happy so far and will update if that changes.",
     "sentiment_id": None},
    {"user_id": None, "product_id": "B09PL7GCHV", "review_id": "R102YA9MC0BT2W", "rating": 5.0,
     "review_text": "Perfect for my 10 year old son to take to school! 😎😎😎", "sentiment_id": None},
    {"user_id": None, "product_id": "B09HZ5GTBP", "review_id": "R104EEVOTRJF88", "rating": 2.0,
     "review_text": "Do not use this charger if you have a dell xps 15 It will fry your computer!!", "sentiment_id": None},
    {"user_id": None, "product_id": "B0BRTGTH8R", "review_id": "R106ADEM54ZA39", "rating": 5.0,
     "review_text": "Came in a nice envelope ready to mail. There is a separate card to write a sentiment that matches. The colors are vibrant and the card is beautiful. It is easy to stand up. I ordered 2 different ones and they are both pretty and sturdy. The price is good for the product received. Also, The cost to mail it was reasonable ($2.35 US domestic). Would recommend.", "sentiment_id": None},
    {"user_id": None, "product_id": "B079DHNK3H", "review_id": "R106S0OOV54GFD", "rating": 3.0,
     "review_text": "Bag shipped damage. When I weight bag was 19.15 lb", "sentiment_id": None},
    {"user_id": None, "product_id": "B0C58N5J9Q", "review_id": "R106VWBNF5NMRI", "rating": 5.0,
     "review_text": "Using for mac pro and working perfectly", "sentiment_id": None},
    {"user_id": None, "product_id": "B0B1CNJL7N", "review_id": "R10B2V81LBFYX0", "rating": 1.0,
     "review_text": "Cheap material, not sturdy at all", "sentiment_id": None},
    {"user_id": None, "product_id": "B0CRPC5432", "review_id": "R10BYE20UOZFNF", "rating": 5.0,
     "review_text": "I am very satisfied with the purchase of my laptop it is compact, light, easy to use and install programs, it has good capacity and can also be expanded, it is a very good option for the price it has. So far I love everything in it", "sentiment_id": None}
]

# Convert data to DataFrame
df = pd.DataFrame(data)

# Handling missing values
df['review_text'].fillna('', inplace=True)

# Preprocessing
# Tokenization and feature extraction using TF-IDF for the 'review_text'
tfidf_vectorizer = TfidfVectorizer(max_features=1000)  # Adjust max_features as needed
X_text = tfidf_vectorizer.fit_transform(df['review_text'])

# Include 'rating' as a feature
X_rating = df['rating'].values.reshape(-1, 1)

# Concatenate the features
X = pd.concat([pd.DataFrame(X_text.toarray()), pd.DataFrame(X_rating)], axis=1)

# Define thresholds for positive, neutral, and negative ratings
positive_threshold = 4
negative_threshold = 2

# Convert 'rating' column to categorical based on thresholds
df['analysis_through_rating'] = pd.cut(df['rating'], bins=[-float('inf'), negative_threshold, positive_threshold, float('inf')],
                                        labels=['Negative', 'Neutral', 'Positive'])

# Target variable for training model on rating
y_rating = df['analysis_through_rating']

# Splitting the data into training and testing sets for rating analysis
X_train_rating, X_test_rating, y_train_rating, y_test_rating = train_test_split(X, y_rating, test_size=0.2, random_state=42)

# Training the model for rating analysis (Logistic Regression example)
classifier_rating = LogisticRegression(max_iter=1000)
classifier_rating.fit(X_train_rating, y_train_rating)

# Predicting sentiment for the entire dataset based on 'rating' column
df['predicted_sentiment_rating'] = classifier_rating.predict(X)

# Training the model for review analysis (Logistic Regression example)
classifier_review = LogisticRegression(max_iter=1000)
classifier_review.fit(X_text, y_rating)  # Using only review text for training

# Predicting sentiment for the entire dataset based on review analysis
df['predicted_sentiment_review'] = classifier_review.predict(X_text)

# Print the DataFrame
print(df)


  user_id  product_id       review_id  rating  \
0    None  B09TRF4LM9  R100AEOXPX3ZP7     5.0   
1    None  B09BG6DSBY  R101PRE3GXWPG7     2.0   
2    None  B08438H4GM  R102U9PYQ6S094     5.0   
3    None  B09PL7GCHV  R102YA9MC0BT2W     5.0   
4    None  B09HZ5GTBP  R104EEVOTRJF88     2.0   
5    None  B0BRTGTH8R  R106ADEM54ZA39     5.0   
6    None  B079DHNK3H  R106S0OOV54GFD     3.0   
7    None  B0C58N5J9Q  R106VWBNF5NMRI     5.0   
8    None  B0B1CNJL7N  R10B2V81LBFYX0     1.0   
9    None  B0CRPC5432  R10BYE20UOZFNF     5.0   

                                         review_text sentiment_id  \
0  wasn't sure about an off-brand, but this charg...         None   
1  These are so slow!! Even with an extra storage...         None   
2  So I've tried several different docks and this...         None   
3  Perfect for my 10 year old son to take to scho...         None   
4  Do not use this charger if you have a dell xps...         None   
5  Came in a nice envelope ready to mail. Ther